In [1]:
import os
import sys
# allow loading modules from local directory.
sys.path.insert(1, '/home/jovyan/work/code')

from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import SERVICE_NAME, Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.instrumentation.langchain import LangchainInstrumentor

# Logging (Experimental)
from opentelemetry._logs import set_logger_provider
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import (
    OTLPLogExporter,
)
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.sdk.resources import Resource

#from otel_grpc import configure_otel_otlp

from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from pipeline import DocumentPipeline
from localrag import LocalRAG

import nest_asyncio
import asyncio 
import logging
 
# Enable nested event loops
nest_asyncio.apply()

# Initialise and Setup OpenTelemetry for the session
resource = Resource(attributes={
  SERVICE_NAME:  os.getenv('OTEL_SERVICE_NAME', 'jupyter-demo')
})
provider = TracerProvider(resource=resource)
processor = BatchSpanProcessor(OTLPSpanExporter(endpoint=os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT")))
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)

 # Configure Logging
#configure_otel_otlp( os.getenv('OTEL_SERVICE_NAME', 'jupyter-demo'), endpoint=os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT"))

logger_provider = LoggerProvider(
    resource=resource
)
set_logger_provider(logger_provider)

exporter = OTLPLogExporter(insecure=True)
logger_provider.add_log_record_processor(BatchLogRecordProcessor(exporter))
handler = LoggingHandler(level=logging.NOTSET, logger_provider=logger_provider)

# Attach OTLP handler to root logger
logging.getLogger().addHandler(handler)

# logging.basicConfig()
#logging.basicConfig(format = "%(asctime)s:%(levelname)s:%(message)s", level = logging.DEBUG)
logging.root.setLevel(logging.INFO)


LangchainInstrumentor().instrument()
logger = logging.getLogger(__name__)
tracer = trace.get_tracer(__name__)


def parse_ollama_connection(conn_str):
    parts = conn_str.split(';')
    endpoint = next(p.split('=')[1] for p in parts if p.startswith('Endpoint='))
    model = next(p.split('=')[1] for p in parts if p.startswith('Model='))
    return endpoint, model

chat_conn = os.getenv('ConnectionStrings__chat-model', 'Endpoint=http://ollama:11434;Model=phi3.5')
chat_model_url, chat_model_id = parse_ollama_connection(chat_conn)

embeddings_conn = os.getenv('ConnectionStrings__embedding-model', 'Endpoint=http://ollama:11434;Model=mxbai-embed-large')
embedding_model_url, embeddings_model = parse_ollama_connection(embeddings_conn)

qdrant_conn = os.getenv('ConnectionStrings__qdrant_http', 'Endpoint=http://qdrant:6334;Key=aMjJKx0t1a6E9hysaCacWz')
parts = qdrant_conn.split(';')
qdrant_url = next(p.split('=')[1] for p in parts if p.startswith('Endpoint='))
qdrant_key = next(p.split('=')[1] for p in parts if p.startswith('Key='))

vector_db_config = VectorDBConfig(
    url=qdrant_url,
    api_key=qdrant_key,
    collection_name="evaluation-ek-md"
)
chat_config = ChatConfig(
    model_name=chat_model_id,
    base_url=chat_model_url
)
embedding_config = EmbeddingConfig(
    model_name=embeddings_model,
    base_url=embedding_model_url
)
processing_config = ProcessingConfig(
    chunk_size=500,
    chunk_overlap=50,
    add_metadata=True,
    extract_code_entities=True
)

openAiKey = os.getenv("OPENAI_KEY")
os.environ["OPENAI_API_KEY"] = openAiKey

In [23]:
import pandas as pd
 
file_name= "test_data_50.pkl" 
test_dataset =  pd.read_pickle(file_name)
# df1

test_dataset.head()
#for index, row in test_dataset.iterrows():
#    print(row["reference_contexts"])
#    reference= row["reference"]

,user_input,reference_contexts,reference,synthesizer_name
0,How can Zoom be effectively utilized to addres...,[Skills and Behaviors of allies To be an effec...,Zoom can be effectively utilized to address is...,single_hop_specifc_query_synthesizer
1,How can corporate sales teams effectively supp...,[Skills and Behaviors of allies To be an effec...,Corporate sales teams can effectively support ...,single_hop_specifc_query_synthesizer
2,Wht is GitLab's role in allyship?,[Tips on being an ally Identifying your power ...,GitLab promotes allyship through initiatives l...,single_hop_specifc_query_synthesizer
3,Wht is the Ally Lab Larning Group and how does...,[Tips on being an ally Identifying your power ...,The Ally Lab Learning Group is an initiative d...,single_hop_specifc_query_synthesizer
4,Wht is GitLab's role in diversity and inclusion?,[What it means to be an ally Take on the strug...,"GitLab provides Diversity, Inclusion & Belongi...",single_hop_specifc_query_synthesizer


In [7]:
pipeline = DocumentPipeline(
    vector_db_config=vector_db_config,
    embedding_config=embedding_config, 
    processing_config=processing_config
)

# ###Process git repository
# repository="https://github.com/dotnet/docs-aspire"
# input_file_name="merged_output.txt"
# with tracer.start_as_current_span(f"Starting ingesting file {input_file_name}"):
#     pipeline.process_single_file(input_file_name,repository)



/home/jovyan/work/code/pipeline.py:59: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)
INFO:httpx:HTTP Request: GET http://qdrant:6333/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://qdrant:6333/collections/evaluation-ek-md/exists "HTTP/1.1 200 OK"
INFO:IngestionPipeline:Collection evaluation-ek-md already exists
INFO:httpx:HTTP Request: GET http://qdrant:6333/collections/evaluation-ek-md "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"


In [28]:
#https://docs.ragas.io/en/latest/getstarted/rag_eval/#basic-setup

evaluation_data=[]

def demonstrate_local_rag(rag, question): 
    references=[]
    with tracer.start_as_current_span("Getting answer and context."): 
        print(f"Question: {question}")
        with tracer.start_as_current_span("rag get context"):
            chunks = rag.get_relevant_chunks(question, k=10)
            for i, chunk in enumerate(chunks, 1):
                references.append(chunk.page_content)
        with tracer.start_as_current_span("Retrieve answers."):
            answer = rag.retrieve_and_answer(question, k=10)
            return (answer, references)

rag = LocalRAG(
    vector_db_config=vector_db_config,
    embedding_config=embedding_config, 
    chat_config=chat_config
)

with tracer.start_as_current_span("Starting demo"):
    for index, row in test_dataset.iterrows():
        print(f"Question {index}:")
        question = row["user_input"]
        reference= row["reference"]
        answer,contexts = demonstrate_local_rag(rag, question)
        print(answer)
        evaluation_data.append({
            "user_input": question,
            "retrieved_contexts": contexts,
            "response": answer,
            "reference": reference
        })
        if(index>=10):
            break

 


/home/jovyan/work/code/localrag.py:46: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)
INFO:httpx:HTTP Request: GET http://qdrant:6333/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://qdrant:6333/collections/evaluation-ek-md "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"


Question 0:
Question: How can Zoom be effectively utilized to address issues of diversity and inclusion within a sales team, particularly when addressing the misuse of pronouns?


INFO:httpx:HTTP Request: POST http://ollama:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"


I cannot find the answer in the provided context. 

Question 1:
Question: How can corporate sales teams effectively support the Black Lives Matter movement?


INFO:httpx:HTTP Request: POST http://ollama:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"


...

Managers can contribute to fostering DIB by:

* **Initiating DIB Roundtables:** Managers can request a DIB Roundtable through the specified issue template. This allows for structured discussions and activities related to DIB. 
* **Leading Open Discussions:** Encourage open communication within their team meetings.  They can start by asking for team members' perspectives on what inclusion means to them, what they find important for feeling included, and creating specific plans around building a more inclusive environment.  
* **Facilitating Activities:** The manager should work with the DIB Team to organize activities that promote deeper engagement in DIB, including workshops or icebreakers focused on the topic. 
* **Encouraging Open Communication:** Creating an open and safe environment where team members feel comfortable sharing their ideas and concerns about DIB is crucial for fostering a sense of belonging.  


By taking these actions, managers can play a significant role in promoti

INFO:httpx:HTTP Request: POST http://ollama:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://ollama:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/evaluation-ek-md/points/query "HTTP/1.1 200 OK"


Managers can contribute to fostering a diverse and inclusive environment for their sales teams in several ways, as outlined in the context provided:

**1.  Initiate DIB Roundtables:**
   * Managers can request a DIB Roundtable with the DIB Team for their team. 
   * This allows the DIB Team to provide guidance on inclusivity best practices specific to sales and the team's unique challenges and opportunities.

**2. Promote Self-Awareness and Privilege Ownership:** 
    *  Managers should encourage open dialogue about diversity, inclusion, and belonging within their teams.
    * When addressing team dynamics (like the "Product Meeting Example"), managers should guide their teams to be aware of any potential bias in decision-making or how they can actively promote a more inclusive environment.

**3. Encourage Actionable Steps:** 
   * Managers need to move beyond simply recognizing issues and take action. For example:
     * If a team meeting lacks diverse perspectives, the manager should

INFO:httpx:HTTP Request: POST http://ollama:11434/api/chat "HTTP/1.1 200 OK"


GitLab leverages its fully remote work model to foster inclusivity in the sales team. By working asynchronously and recording videos, GitLab allows for greater participation from team members around the world with diverse cultural backgrounds and experiences. This promotes a more equitable environment where everyone can contribute their unique skillset and perspectives. 

The context emphasizes that this approach helps redress imbalances by providing underrepresented groups with opportunities to gain visibility and leadership roles within the sales department, leading to better decisions and greater team member belonging.  



In [30]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas import EvaluationDataset

llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()
evaluation_dataset = EvaluationDataset.from_list(evaluation_data)
evaluator_llm = LangchainLLMWrapper(llm)
result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
print(result)


Evaluating:   0%|          | 0/33 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

{'context_recall': 0.7338, 'faithfulness': 0.8564, 'factual_correctness': 0.3160}
